# 🔗 Integração de Dados, Imagens e LIME

As células abaixo carregam dados, exibem imagens e geram uma explicação com **LIME** (se instalado).

In [ ]:

# 1) Carregar dados de exemplo
import pandas as pd
from pathlib import Path

dados_csv = Path("dados") / "clientes_exemplo.csv"
df = pd.read_csv(dados_csv)
display(df.head())

print(f"Arquivo carregado: {dados_csv.resolve()}")
print(f"Formato: {df.shape[0]} linhas x {df.shape[1]} colunas")


In [ ]:

# 2) Visualização básica (dispersão) e exibição de imagem salva
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
from pathlib import Path

# Dispersão
plt.figure()
plt.scatter(df["idade"], df["salario"], c=df["comprou"], edgecolors="k")
plt.xlabel("Idade")
plt.ylabel("Salário")
plt.title("Distribuição de Clientes (Exemplo)")
plt.show()

# Exibir imagem salva
img_path = Path("imagens") / "grafico_clientes.png"
if img_path.exists():
    img = mpimg.imread(img_path)
    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.title("Gráfico salvo: imagens/grafico_clientes.png")
    plt.show()
else:
    print("Imagem não encontrada:", img_path)


In [ ]:

# 3) Exemplo de treino simples + LIME (com fallback se LIME não estiver instalado)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

X = df[["idade", "salario"]].values
y = df["comprou"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression())
])
pipe.fit(X_train, y_train)
print("Acurácia (hold-out):", pipe.score(X_test, y_test))

# Tentar usar LIME
try:
    from lime.lime_tabular import LimeTabularExplainer
    explainer = LimeTabularExplainer(
        X_train,
        feature_names=["idade", "salario"],
        class_names=["nao_comprou", "comprou"],
        discretize_continuous=True
    )
    i = 0  # explicar a primeira instância do conjunto de teste
    exp = explainer.explain_instance(
        X_test[i],
        pipe.predict_proba,
        num_features=2
    )
    print("Explicação (lista de pesos):")
    for feat, weight in exp.as_list():
        print(f"  {feat}: {weight:.4f}")
except ImportError as e:
    print("LIME não está instalado. Para instalar, execute no seu ambiente:")
    print("  pip install lime")
except Exception as e:
    print("Não foi possível gerar a explicação com LIME:", e)


_Última atualização automática:_ **2025-08-08 20:45:42**